<a href="https://colab.research.google.com/github/evaalonsoortiz/dyn-rt-shim-sim/blob/main/dyn_rt_shim_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Realtime z-Shimming

This Colab notebook contains the simulation and analysis presented in xxx.




## Setup

To setup, we first clone the dyn_rt_shim_sim and spinalcordtoolbox repos and then instal the spinalcordtoolbox.

In [4]:
 ! git clone https://github.com/evaalonsoortiz/dyn-rt-shim-sim

% cd dyn-rt-shim-sim
! pip install -r requirements.txt

Cloning into 'dyn-rt-shim-sim'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 1), reused 10 (delta 1), pack-reused 0
Unpacking objects: 100% (10/10), done.
/content/dyn-rt-shim-sim
     |████████████████████████████████| 3.1MB 8.7MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-cp37-none-any.whl size=59374 sha256=00e3faf53a3e9923c2439415ae940773013dd9f3970bc281f6b32f25b5fc1329
  Stored in directory: /root/.cache/pip/wheels/3d/3c/84/28d36677f3c760c048bd02b5a547ea0c4027770cc9cdb9af1e
Successfully built transforms3d


In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
import nibabel as nib
import sys
import os
import cv2
import re
import pandas as pd
import math
from os.path import join
from scipy.ndimage import gaussian_filter

## Load Images

In [6]:
sc_mask = nib.load('vol0015.nii')
img = nib.load('GRE-T1W_0019_GRE-T1w_20210407174936_19_slice_0015.nii')

img_data = img.get_fdata()
sc_mask_data = sc_mask.get_fdata()

## Define Relevant Constants

In [7]:
# -----------------------------------------------------------------------------#
# image acquistion parameters
# -----------------------------------------------------------------------------#
matrix = img.shape
image_res = img.header.get_zooms()

fov = np.array(image_res) * np.array(matrix)

TE = 15e-3 # [s]
TR = 1000e-3 # [s]

# -----------------------------------------------------------------------------#
# define k-space constants
# -----------------------------------------------------------------------------#
k_max = 1 / ( 2 * np.array(image_res) ) # maximum measured spatial frequencies in units of mm^-1
delta_k = 1 / fov # distance between points in k-space in units of mm^-1
[kx,ky] = np.meshgrid( np.linspace(start=-k_max[0],stop=k_max[0],num=matrix[0]) , np.linspace(start=-k_max[1],stop=k_max[1],num=matrix[1]), indexing='ij' )

# -----------------------------------------------------------------------------#
# define other constants related to the time-varying magnetic field
# -----------------------------------------------------------------------------#
w_r = 2 * math.pi / 6 # radial frequency of the respiratory motion in units of [rad/s]
RIROmax_uniform = 20 # maximum frequency offset caused by respiration (during inhalation) in units of [Hz]

# -----------------------------------------------------------------------------#
# define the spatial distribution for RIROmax
# -----------------------------------------------------------------------------#
[kx,ky] = np.meshgrid( np.linspace(start=-k_max[0],stop=k_max[0],num=matrix[0]) , np.linspace(start=-k_max[1],stop=k_max[1],num=matrix[1]), indexing='ij' )
[x,y] = np.meshgrid( np.linspace(start=-(matrix[0]-1)/2,stop=(matrix[0]-1)/2,num=matrix[0]) , np.linspace(start=-(matrix[1]-1)/2,stop=(matrix[1]-1)/2,num=matrix[1]), indexing='ij' ) # image grid (in [mm])

r = np.sqrt( (x*image_res[0])**2 + (y*image_res[1])**2 ) # radial position (in [mm])
r = abs( (r - np.max(np.max(r)) ) / np.max(np.max(r)) )
r = r**3;
RIROmax = RIROmax_uniform*r

# -----------------------------------------------------------------------------#
# limit the RIROmax distribution to where you have signal in your image
# -----------------------------------------------------------------------------#

# Prepare noise mask
noise_mask = np.zeros(matrix)

# Pick four corners of 5x5
noise_mask[0:4,0:4] = 1
noise_mask[0:4,(-1-4):-1] = 1
noise_mask[(-1-4):-1,0:4] = 1
noise_mask[(-1-4):-1,(-1-4):-1] = 1

# apply mask to the data 
noise_data = np.multiply(img_data, noise_mask)

# Calculate background noise 
sigma = noise_data[noise_data!=0].std()

# create background mask
bkgrnd_mask = np.zeros(matrix)
bkgrnd_mask[img_data>(15*sigma)] = 1;


RIROmax = np.multiply(bkgrnd_mask,RIROmax)
RIROmax = gaussian_filter(RIROmax, sigma=2)



## Simulate the reconstructed image in the presence of a time-varying magnetic field

In [54]:
# k-space data for the ideal image
FFT = np.fft.fftshift(np.fft.fft(np.fft.fftshift(img_data)))

# modify the k-space data to take into account the effect of B(z,t)
FFT_mod = np.multiply( np.exp( (-2 * math.pi * 1j * TE) * RIROmax * np.sin(w_r * (TR * (matrix[1]/2 + np.divide(ky,delta_k[1]) ) + TE ) ) ) , FFT ) 

#recontruct the image
calcImage = np.fft.ifftshift(np.fft.ifft(np.fft.ifftshift(FFT_mod)))

# display
fig = px.imshow(np.rot90(np.abs(calcImage)), color_continuous_scale='gray', zmin=0, zmax=1500)
fig.update_layout(
    title_text='Simualted measurement, RIROmax(x,y)'
)
fig.show()
